In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from functools import reduce

pd.set_option('display.max_rows', 10000)

In [2]:
def combine_indicators(currency, pair):
    
    cpi = pd.read_csv("../data/processed/cpi/{}_cpi_processed.csv".format(currency))
    gdp = pd.read_csv("../data/processed/gdp/{}_gdp_processed.csv".format(currency))
    ir = pd.read_csv("../data/processed/interest_rate/{}_ir_processed.csv".format(currency))
    ppi = pd.read_csv("../data/processed/ppi/{}_ppi_processed.csv".format(currency))
    ue = pd.read_csv("../data/processed/unemployment_rate/{}_ue_processed.csv".format(currency))
    news = pd.read_csv("../data/processed/news/news_sentiment.csv")
    news = news[{"Time", currency.upper()}]
    news = news.rename(columns={currency.upper(): "News Sentiment"})
    tweets = pd.read_csv("../data/processed/tweets/tweets_sentiment.csv")
    tweets = tweets[{"Time", currency.upper()}]
    tweets = tweets.rename(columns={currency.upper(): "Twitter Sentiment"})
    
    combined_df = merge_dataframe([cpi, gdp, ir, ppi, ue, news, tweets])
    
    if currency.upper() in pair:
        exchange_rate = pd.read_csv("../data/processed/exchange_rate/{}_exchange.csv".format(pair))
        combined_df = merge_dataframe([combined_df, exchange_rate])
            
    combined_df = combined_df[combined_df["RSI"].notnull()]
    
    return combined_df
    
def merge_dataframe(data_list):
    merged_data = reduce(lambda left, right : pd.merge(left, right, how="outer", on="Time"), data_list)     
    merged_data.sort_values(by=["Time"], inplace=True)
    merged_data = merged_data.reset_index(drop=True)
    return merged_data

def currency_pair(buy, sell):
    pair = (buy + sell).upper()
    buy_df = combine_indicators(buy, pair)
    buy = buy.upper()
    buy_df = buy_df.rename(columns={
            "CPI": buy + "_CPI", 
            "GDP": buy + "_GDP", 
            "Interest Rate": buy + " Interest Rate",
            "PPI": buy + "_PPI",
            "Unemployment Rate": buy + " Unemployment Rate",
            "News Sentiment": buy + " News Sentiment",
            "Twitter Sentiment": buy + " Twitter Sentiment",
        })
    buy_df = buy_df.reset_index(drop=True)
    sell_df = combine_indicators(sell, pair)
    sell_df = sell_df[{"Time", "CPI", "GDP", "Interest Rate", "PPI", "Unemployment Rate", "News Sentiment", "Twitter Sentiment"}]
    sell = sell.upper()
    sell_df = sell_df.rename(columns={
            "CPI": sell + "_CPI", 
            "GDP": sell + "_GDP", 
            "Interest Rate": sell + " Interest Rate",
            "PPI": sell + "_PPI",
            "Unemployment Rate": sell + " Unemployment Rate",
            "News Sentiment": sell + " News Sentiment",
            "Twitter Sentiment": sell + " Twitter Sentiment",
        })
    sell_df.reset_index(drop=True)
    pair_df = buy_df.merge(sell_df, how="inner", on=["Time"])
    pair_df['Time'] = pd.to_datetime(pair_df['Time'], utc=True)
    cool = pd.read_csv("../data/external/exchange_rates/EURUSD_M1.csv")
    cool = cool.rename(columns={"DateTime": "Time", "Close": "Real Close"})
    cool = cool[{'Time', 'Real Close'}]
    cool = pd.DataFrame(cool, columns=['Time', 'Real Close'])
    cool = convert_date(cool)
    cool['Time'] = pd.to_datetime(cool['Time'], utc=True)
    print(cool)
    pair_df = pair_df.merge(cool, how="left", on=["Time"])
    print(pair_df)
    pair_df = configure_time(15, pair_df)
    pair_df.to_csv("../data/processed/{}_processed.csv".format(pair), index=False)
    return pair_df

def configure_time(minutes, dataframe):
    dataframe["Volume"] = dataframe["Volume"].rolling(minutes, min_periods=1).sum()
    dataframe["High"] = dataframe["High"].rolling(minutes, min_periods=1).max()
    dataframe["Low"] = dataframe["Low"].rolling(minutes, min_periods=1).min()
    
    time_frame = pd.date_range(start="2018-01-01 22:00:00", freq="{}T".format(minutes), end="2020-12-31 21:59:00")
    time_frame = pd.DataFrame(time_frame, columns=["Time"])
    time_frame["Time"] = time_frame["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    time_frame['Time'] = pd.to_datetime(time_frame['Time'], utc=True)
    
    configured_df = time_frame.merge(dataframe, how="inner", on="Time")
    configured_df.at[0, "Volume"] = configured_df.at[1, "Volume"]
    configured_df["Open"] = configured_df["Open"].shift(1)
    configured_df.at[0, "Open"] = configured_df.at[1, "Open"]
    
    return configured_df

def convert_date(exchange):
    exchange["Time"] = pd.to_datetime(exchange["Time"], format="%Y-%m-%d %H:%M:%S")
    return exchange

In [3]:
pair = currency_pair("eur", "usd")
pair

                             Time  Real Close
0       2017-01-01 22:00:00+00:00     1.05153
1       2017-01-01 22:01:00+00:00     1.05153
2       2017-01-01 22:02:00+00:00     1.05175
3       2017-01-01 22:03:00+00:00     1.05175
4       2017-01-01 22:08:00+00:00     1.05170
...                           ...         ...
1523482 2021-02-01 23:55:00+00:00     1.20677
1523483 2021-02-01 23:56:00+00:00     1.20677
1523484 2021-02-01 23:57:00+00:00     1.20681
1523485 2021-02-01 23:58:00+00:00     1.20678
1523486 2021-02-01 23:59:00+00:00     1.20676

[1523487 rows x 2 columns]
                             Time  EUR_CPI   EUR_GDP  EUR Interest Rate  \
0       2018-01-01 22:01:00+00:00      1.3  0.737765             1.0339   
1       2018-01-01 22:02:00+00:00      1.3  0.737765             1.0339   
2       2018-01-01 22:03:00+00:00      1.3  0.737765             1.0339   
3       2018-01-01 22:04:00+00:00      1.3  0.737765             1.0339   
4       2018-01-01 22:05:00+00:00      1.3  0

,Time,EUR_CPI,EUR_GDP,EUR Interest Rate,EUR_PPI,EUR Unemployment Rate,EUR News Sentiment,EUR Twitter Sentiment,Volume,Open,...,RSI,A/D Index,USD Interest Rate,USD_GDP,USD Twitter Sentiment,USD_CPI,USD_PPI,USD News Sentiment,USD Unemployment Rate,Real Close
0,2018-01-01 22:15:00+00:00,1.3,0.737765,1.0339,0.005859,8.6,0.0,0.0,570.0,0.000025,...,55.486755,-57.000000,2.58,0.565669,0.000000,2.070508,0.012921,0.000000,4.0,1.20062
1,2018-01-01 22:30:00+00:00,1.3,0.737765,1.0339,0.005859,8.6,0.0,0.0,570.0,0.000025,...,61.542144,20.705882,2.58,0.565669,0.000000,2.070508,0.012921,0.000000,4.0,1.20076
2,2018-01-01 22:45:00+00:00,1.3,0.737765,1.0339,0.005859,8.6,0.0,0.0,394.0,0.000050,...,72.855107,0.000000,2.58,0.565669,0.000000,2.070508,0.012921,0.000000,4.0,1.20139
3,2018-01-01 23:00:00+00:00,1.3,0.737765,1.0339,0.005859,8.6,0.0,0.0,512.0,0.000000,...,45.046777,-103.000000,2.58,0.565669,0.000000,2.070508,0.012921,0.000000,4.0,1.20120
4,2018-01-01 23:15:00+00:00,1.3,0.737765,1.0339,0.005859,8.6,0.0,0.0,2540.0,-0.000025,...,61.860883,-25.333333,2.58,0.565669,0.000000,2.070508,0.012921,0.000000,4.0,1.20147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74711,2020-12-31 20:45:00+00:00,-0.3,-0.697576,-0.0913,0.001949,8.3,0.0,0.0,550.0,-0.000057,...,44.066916,-27.000000,0.93,0.988378,-0.001073,1.362005,0.022647,0.000128,6.7,1.22156
74712,2020-12-31 21:00:00+00:00,-0.3,-0.697576,-0.0913,0.001949,8.3,0.0,0.0,898.0,-0.000008,...,46.002475,-26.173913,0.93,0.988378,-0.000852,1.362005,0.022647,0.000128,6.7,1.22151
74713,2020-12-31 21:15:00+00:00,-0.3,-0.697576,-0.0913,0.001949,8.3,0.0,0.0,862.0,-0.000098,...,67.353758,-212.000000,0.93,0.988378,-0.000852,1.362005,0.022647,0.000128,6.7,1.22242
74714,2020-12-31 21:30:00+00:00,-0.3,-0.697576,-0.0913,0.001949,8.3,0.0,0.0,906.0,0.000294,...,54.013982,76.142857,0.93,0.988378,-0.000852,1.362005,0.022647,0.000128,6.7,1.22210
